In [7]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2

# Define the path to the directory containing the Mel spectrogram images
data_dir = 'F:\\FYP\\Dataset\\baum1s\\Mel_Spectrograms'

# Define the emotion labels
emo_dict = {"happiness": 0, "anger": 1, "sadness": 2, "disgust": 3, "fear": 4, "interest": 5, "unsure": 6}

# Initialize empty lists for the images and labels
X = []
y = []

# Loop through each emotion label and load the corresponding images
for emo_label in emo_dict.keys():
    emo_dir = os.path.join(data_dir, emo_label)
    for img_file in os.listdir(emo_dir):
        img_path = os.path.join(emo_dir, img_file)
        img = plt.imread(img_path)
        #img = cv2.resize(img, (256,256))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        X.append(img)
        y.append(emo_dict[emo_label])

print("Number of images loaded: ", len(X))


Number of images loaded:  1390


In [8]:
# Convert the lists to numpy arrays
X = np.array(X)
y = np.array(y)


In [9]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

Shape of X:  (1390, 448, 448, 3)
Shape of y:  (1390,)


In [10]:
# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the shape of each set
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Testing set:", X_test.shape, y_test.shape)

Training set: (889, 448, 448, 3) (889,)
Validation set: (223, 448, 448, 3) (223,)
Testing set: (278, 448, 448, 3) (278,)


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the VGG16 model with pre-trained ImageNet weights, excluding the top layer
base_model = tf.keras.applications.VGG16(input_shape=(448, 448, 3), include_top=False, weights="imagenet")

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add a custom top layer for emotion recognition
inputs = keras.Input(shape=(448, 448, 3))
x = base_model(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(7, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=["accuracy"])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

Epoch 1/10
28/28 [==============================] - 909s 32s/step - loss: 4.5916 - accuracy: 0.2047 - val_loss: 1.8595 - val_accuracy: 0.2197
Epoch 2/10
28/28 [==============================] - 952s 34s/step - loss: 1.9018 - accuracy: 0.2486 - val_loss: 1.8404 - val_accuracy: 0.2242
Epoch 3/10
28/28 [==============================] - 941s 34s/step - loss: 1.8830 - accuracy: 0.2553 - val_loss: 1.8210 - val_accuracy: 0.2601
Epoch 4/10
28/28 [==============================] - 910s 33s/step - loss: 1.9052 - accuracy: 0.2373 - val_loss: 1.8154 - val_accuracy: 0.3004
Epoch 5/10
28/28 [==============================] - 896s 32s/step - loss: 1.8647 - accuracy: 0.2306 - val_loss: 1.8145 - val_accuracy: 0.3184
Epoch 6/10
28/28 [==============================] - 903s 32s/step - loss: 1.8435 - accuracy: 0.2520 - val_loss: 1.8102 - val_accuracy: 0.3498
Epoch 7/10
28/28 [==============================] - 916s 33s/step - loss: 1.8335 - accuracy: 0.2486 - val_loss: 1.8497 - val_accuracy: 0.3139
Epoch 